In [1]:
#ejecutar solo la primera vez para instalar las dependencias

#!python ./get-pip.py
#!pip install transformers
#!pip install torch
#!pip install pandas
#!pip install fastparquet
#!pip install --upgrade pip
#!pip install accelerate>=0.20.1
#!pip install transformers[torch]
#!pip install scikit-learn
#!pip install accelerate -U
#!pip install tensorflow
#!pip install datasets
#!pip install rouge-score


In [2]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForQuestionAnswering, TrainingArguments, Trainer
import torch
from sklearn.model_selection import train_test_split
from datasets import load_metric

In [5]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('data3.csv')

# Extraer las preguntas y respuestas del DataFrame
questions = df['query'].tolist()
positives = df['positive'].tolist()
context = 'Quickfood, una aplicación versátil y eficiente, brinda soluciones a diversos usuarios en el ámbito de eventos y gastronomía. ¿Te preguntas si los Encargados de Puestos de Comida pueden actualizar su menú en tiempo real? ¡Claro! Quickfood les ofrece esta capacidad, permitiéndoles personalizar y variar su oferta para satisfacer las preferencias de los Consumidores. Si eres Productor de Eventos, te preguntarás si puedes gestionar múltiples puestos de comida a través de Quickfood. ¡La respuesta es sí! La aplicación te brinda la capacidad de coordinar y administrar eficientemente varios puestos para eventos masivos o privados. ¿Te preocupa la privacidad de los Consumidores? Quickfood garantiza la seguridad al permitir a los Repartidores acceder a la ubicación precisa solo para realizar entregas directas a sus asientos o lugares designados. Si te preguntas si Quickfood funciona en Android, la respuesta es sí. ¡La aplicación es compatible tanto con dispositivos Android como con navegadores web para una mayor accesibilidad! Para los Consumidores, Quickfood ofrece la comodidad de realizar pedidos desde cualquier asiento en el evento, brindando una experiencia eficiente y cómoda. Además, se permite realizar un seguimiento en tiempo real de los pedidos, desde la realización hasta la entrega. Los Encargados de Puestos de Comida pueden gestionar pagos y transacciones fácilmente a través de Quickfood, facilitando una experiencia integrada y conveniente. Además, los Productores de Eventos pueden visualizar el estado y congestión de los pedidos en tiempo real. En cuanto a los métodos de pago, Quickfood acepta Mercado Pago para transacciones seguras y convenientes. Para Repartidores, la aplicación proporciona información detallada sobre los pedidos pendientes y las ubicaciones de entrega. Si te interesa convertirte en Repartidor, Quickfood te permite hacerlo registrándote primero como Consumidor y cumpliendo con los requisitos adicionales para ser Repartidor. Asimismo, para aquellos que deseen trabajar para Quickfood, ¡la respuesta es sí! Quickfood se adapta a diversas situaciones, permitiendo a los usuarios personalizar menús para eventos específicos, ya sea que sean Productores de Eventos, Encargados de Puestos de Comida o Consumidores. La aplicación también se preocupa por la situación de eventos imprevistos, ofreciendo reembolsos en caso de suspensión o cancelación. Además, proporciona créditos equivalentes a precompras para eventos suspendidos, demostrando su compromiso con los usuarios. Para garantizar la seguridad de la información del usuario, Quickfood emplea medidas de encriptación avanzadas y protocolos de seguridad. Además, cuenta con un equipo de soporte técnico para resolver cualquier inconveniente o inquietud de los usuarios.'

# Tokenización de las preguntas
tokenizer = BertTokenizer.from_pretrained('MMG/bert-base-spanish-wwm-cased-finetuned-squad2-es')

encodings = tokenizer(questions,positives, truncation=True, padding=True)

# Carga del modelo BERT basado en PyTorch
model = BertForQuestionAnswering.from_pretrained('MMG/bert-base-spanish-wwm-cased-finetuned-squad2-es')

In [6]:
start_positions = []
end_positions = []

for question,positive in zip(questions,positives):
    inputs = tokenizer(question,positive, truncation=True, padding=True, return_tensors="pt")

    output = model(**inputs)
    start_logits = output.start_logits
    end_logits = output.end_logits


    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    start_positions.append(start_index.item())
    end_positions.append(end_index.item())


In [7]:
class CustomQADataset(torch.utils.data.Dataset):
    def __init__(self, questions, positives, start_positions, end_positions, tokenizer, max_length):
        self.questions = questions
        self.positives = positives
        self.context = context
        self.start_positions = start_positions
        self.end_positions = end_positions
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        positive = self.positives[idx]

        encoding = self.tokenizer(
            question,
            context,
            positive,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor(self.start_positions[idx]),
            'end_positions': torch.tensor(self.end_positions[idx])
        }

In [8]:
# Dividir los datos en conjuntos de entrenamiento y evaluación
train_questions, eval_questions, train_positives, eval_positives, train_start_positions, eval_start_positions, train_end_positions, eval_end_positions = train_test_split(
    questions, positives, start_positions, end_positions, test_size=0.2, random_state=42)

# Crear conjuntos de datos
train_dataset = CustomQADataset(train_questions, train_positives, train_start_positions, train_end_positions, tokenizer, max_length=512)
eval_dataset = CustomQADataset(eval_questions, eval_positives, eval_start_positions, eval_end_positions, tokenizer, max_length=512)

# Configuración de los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    evaluation_strategy="steps",
    save_steps=500,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=5,
    save_total_limit=2,
    load_best_model_at_end=True,
    per_device_eval_batch_size=8,
)

# Configuración del entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [9]:
trainer.train()

  0%|          | 0/70 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

KeyboardInterrupt: 

In [ ]:
trainer.save_model()

In [ ]:
trainer.evaluate()


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 3.8414316177368164,
 'eval_runtime': 13.3992,
 'eval_samples_per_second': 0.97,
 'eval_steps_per_second': 0.149,
 'epoch': 10.0}